# Testing segmentation based IQMs, using the ones from MRIQC
## Load a segmentation map

In [ ]:
from pathlib import Path
import numpy as np
import nibabel as ni
base = Path("/home/tsanchez/Documents/mial/repositories/qc_fetal_brain/fetal_brain_qc/cli/test/")
sub="003"
run=5


path_seg_proba = base/f"sub-{sub}/ses-01/sub-{sub}_ses-01_acq-haste_run-{run}desc-proba_seg.npz"
path_data = base/f"sub-{sub}/ses-01/sub-{sub}_ses-01_acq-haste_run-{run}_desc-cropped_T2w.nii.gz"
path_seg = base/f"sub-{sub}/ses-01/sub-{sub}_ses-01_acq-haste_run-{run}_seg.nii.gz"

In [ ]:
proba = np.load(path_seg_proba)["probabilities"]
datanii = ni.load(path_data)
data = datanii.get_fdata().transpose(2,1,0)
seg = ni.load(path_seg).get_fdata().transpose(2,1,0)

Defining segmentations according to FETA labels
0. Background and non-brain tissue
1. Cerebrospinal fluid
2. Grey Matter
3. White Matter
4. Ventricles
5. Cerebellum
6. Deep Grey Matter
7. Brainstem

In [ ]:
SEGM = {"BG":0,"CSF":1,"GM":2,"WM":3}#,"LV":4,"CBM":5,"SGM":6,"BS":7}
segm_names = list(SEGM.keys())

Merge data from seg and proba.
- Under GM, put the GM and the SGM (subcortical gray matter)
- Under CSF, put the CSF and ventricle

In [ ]:
print((seg==6).sum(),(seg==4).sum(),(proba[6]).sum(),(proba[4]).sum())

In [ ]:
print((seg==2).sum(),(seg==1).sum(),(proba[2]).sum(),(proba[1]).sum())

In [ ]:
proba[1] += proba[4]
proba[2] += proba[6]
seg[seg==4] = 1 
seg[seg==6] = 2

In [ ]:
print((seg==2).sum(),(seg==1).sum(),(proba[2]).sum(),(proba[1]).sum())

In [ ]:
from scipy.stats import kurtosis

In [ ]:
def summary_stats(data, pvms, airmask=None, erode=True):
    r"""
    Estimates the mean, the median, the standard deviation,
    the kurtosis,the median absolute deviation (mad), the 95\%
    and the 5\% percentiles and the number of voxels (summary\_\*\_n)
    of each tissue distribution.
    .. warning ::
        Sometimes (with datasets that have been partially processed), the air
        mask will be empty. In those cases, the background stats will be zero
        for the mean, median, percentiles and kurtosis, the sum of voxels in
        the other remaining labels for ``n``, and finally the MAD and the
        :math:`\sigma` will be calculated as:
        .. math ::
            \sigma_\text{BG} = \sqrt{\sum \sigma_\text{i}^2}
    """
    from statsmodels.stats.weightstats import DescrStatsW
    from statsmodels.robust.scale import mad

    output = {}
    for label, probmap in pvms.items():
        wstats = DescrStatsW(data=data.reshape(-1), weights=probmap.reshape(-1))
        nvox = probmap.sum()
        p05, median, p95 = wstats.quantile(
            np.array([0.05, 0.50, 0.95]),
            return_pandas=False,
        )
        thresholded = data[probmap > (0.5 * probmap.max())]

        output[label] = {
            "mean": float(wstats.mean),
            "median": float(median),
            "p95": float(p95),
            "p05": float(p05),
            "k": float(kurtosis(thresholded)),
            "stdv": float(wstats.std),
            "mad": float(mad(thresholded, center=median)),
            "n": float(nvox),
        }

    return output

In [ ]:
def volume_fraction(pvms):
    r"""
    Computes the :abbr:`ICV (intracranial volume)` fractions
    corresponding to the (partial volume maps).
    .. math ::
        \text{ICV}^k = \frac{\sum_i p^k_i}{\sum\limits_{x \in X_\text{brain}} 1}
    :param list pvms: list of :code:`numpy.ndarray` of partial volume maps.
    """
    tissue_vfs = {}
    total = 0
    for k, lid in list(SEGM.items()):
        if lid == 0:
            continue
        tissue_vfs[k] = pvms[lid - 1].sum()
        total += tissue_vfs[k]

    for k in list(tissue_vfs.keys()):
        tissue_vfs[k] /= total
    return {k: float(v) for k, v in list(tissue_vfs.items())}

In [ ]:
from math import sqrt


In [ ]:
def snr(mu_fg, sigma_fg, n):
    r"""
    Calculate the :abbr:`SNR (Signal-to-Noise Ratio)`.
    The estimation may be provided with only one foreground region in
    which the noise is computed as follows:
    .. math::
        \text{SNR} = \frac{\mu_F}{\sigma_F\sqrt{n/(n-1)}},
    where :math:`\mu_F` is the mean intensity of the foreground and
    :math:`\sigma_F` is the standard deviation of the same region.
    :param float mu_fg: mean of foreground.
    :param float sigma_fg: standard deviation of foreground.
    :param int n: number of voxels in foreground mask.
    :return: the computed SNR
    """
    return float(mu_fg / (sigma_fg * sqrt(n / (n - 1))))

In [ ]:
def cnr(mu_wm, mu_gm, sigma_air, sigma_wm, sigma_gm):
    r"""
    Calculate the :abbr:`CNR (Contrast-to-Noise Ratio)` [Magnota2006]_.
    Higher values are better.
    .. math::
        \text{CNR} = \frac{|\mu_\text{GM} - \mu_\text{WM} |}{\sqrt{\sigma_B^2 +
        \sigma_\text{WM}^2 + \sigma_\text{GM}^2}},
    where :math:`\sigma_B` is the standard deviation of the noise distribution within
    the air (background) mask.
    :param float mu_wm: mean of signal within white-matter mask.
    :param float mu_gm: mean of signal within gray-matter mask.
    :param float sigma_air: standard deviation of the air surrounding the head ("hat" mask).
    :param float sigma_wm: standard deviation within white-matter mask.
    :param float sigma_gm: standard within gray-matter mask.
    :return: the computed CNR
    """
    # Does this make sense to implement this given that sigma_air=0 artificially?
    return float(abs(mu_wm - mu_gm) / sqrt(sigma_air**2 + sigma_gm**2 + sigma_wm**2))

In [ ]:
def cjv(mu_wm, mu_gm, sigma_wm, sigma_gm):
    r"""
    Calculate the :abbr:`CJV (coefficient of joint variation)`, a measure
    related to :abbr:`SNR (Signal-to-Noise Ratio)` and
    :abbr:`CNR (Contrast-to-Noise Ratio)` that is presented as a proxy for
    the :abbr:`INU (intensity non-uniformity)` artifact [Ganzetti2016]_.
    Lower is better.
    .. math::
        \text{CJV} = \frac{\sigma_\text{WM} + \sigma_\text{GM}}{|\mu_\text{WM} - \mu_\text{GM}|}.
    :param float mu_wm: mean of signal within white-matter mask.
    :param float mu_gm: mean of signal within gray-matter mask.
    :param float sigma_wm: standard deviation of signal within white-matter mask.
    :param float sigma_gm: standard deviation of signal within gray-matter mask.
    :return: the computed CJV
    """
    return float((sigma_wm + sigma_gm) / abs(mu_wm - mu_gm))

In [ ]:
def wm2max(img, mu_wm):
    r"""
    Calculate the :abbr:`WM2MAX (white-matter-to-max ratio)`,
    defined as the maximum intensity found in the volume w.r.t. the
    mean value of the white matter tissue. Values close to 1.0 are
    better:
    .. math ::
        \text{WM2MAX} = \frac{\mu_\text{WM}}{P_{99.95}(X)}
    """
    return float(mu_wm / np.percentile(img.reshape(-1), 99.95))

In [ ]:


        # FBER
        self._results["fber"] = fber(inudata, headdata, rotdata)

        # EFC
        self._results["efc"] = efc(inudata, rotdata)

        

        # Artifacts
        self._results["qi_1"] = art_qi1(airdata, artdata)



        # FWHM
        fwhm = np.array(self.inputs.in_fwhm[:3]) / np.array(imnii.header.get_zooms()[:3])
        self._results["fwhm"] = {
            "x": float(fwhm[0]),
            "y": float(fwhm[1]),
            "z": float(fwhm[2]),
            "avg": float(np.average(fwhm)),
        }



        # Image specs
        self._results["size"] = {
            "x": int(inudata.shape[0]),
            "y": int(inudata.shape[1]),
            "z": int(inudata.shape[2]),
        }
        self._results["spacing"] = {
            i: float(v) for i, v in zip(["x", "y", "z"], imnii.header.get_zooms()[:3])
        }

        try:
            self._results["size"]["t"] = int(inudata.shape[3])
        except IndexError:
            pass

        try:
            self._results["spacing"]["tr"] = float(imnii.header.get_zooms()[3])
        except IndexError:
            pass


In [ ]:
def rpve(pvms, seg):
    """
    Computes the :abbr:`rPVe (residual partial voluming error)`
    of each tissue class.
    .. math ::
        \\text{rPVE}^k = \\frac{1}{N} \\left[ \\sum\\limits_{p^k_i \
\\in [0.5, P_{98}]} p^k_i + \\sum\\limits_{p^k_i \\in [P_{2}, 0.5)} 1 - p^k_i \\right]
    """

    pvfs = {}
    for k, lid in list(SEGM.items()):
        if lid == 0:
            continue
        pvmap = pvms[lid - 1]
        pvmap[pvmap < 0.0] = 0.0
        pvmap[pvmap >= 1.0] = 1.0
        totalvol = np.sum(pvmap > 0.0)
        upth = np.percentile(pvmap[pvmap > 0], 98)
        loth = np.percentile(pvmap[pvmap > 0], 2)
        pvmap[pvmap < loth] = 0
        pvmap[pvmap > upth] = 0
        pvfs[k] = (pvmap[pvmap > 0.5].sum() + (1.0 - pvmap[pvmap <= 0.5]).sum()) / totalvol
    return {k: float(v) for k, v in list(pvfs.items())}

In [ ]:
seg[2].sum()

In [ ]:
proba.shape

In [ ]:
pvms = {segm_names[l]: proba[l] for l in SEGM.values()}
segm_map = {segm_names[l]: seg == l for l in SEGM.values()}
pvmdata = list(pvms.values())
segmdata = list(segm_map.values())

In [ ]:
def eval_iqm(datanii, segm, segm2):
    data = datanii.get_fdata().transpose(2,1,0)
    segmdata = list(segm.values())
    results = {}
    stats = summary_stats(data,segm)
    results["stats"] = stats
    snrvals = []
    results["snr"] = {}
    for tlabel in SEGM.keys():
        snrvals.append(
            snr(
                stats[tlabel]["median"],
                stats[tlabel]["stdv"],
                stats[tlabel]["n"],
            )
        )
        results["snr"][tlabel] = snrvals[-1]
    results["snr"]["total"] = float(np.mean(snrvals))
        # CJV
    results["cjv"] = cjv(
            # mu_wm, mu_gm, sigma_wm, sigma_gm
            stats["WM"]["median"],
            stats["GM"]["median"],
            stats["WM"]["mad"],
            stats["GM"]["mad"],
        )    
    # CNR
    results["cnr"] = cnr(
        stats["WM"]["median"],
        stats["GM"]["median"],
        stats["BG"]["stdv"],
        stats["WM"]["stdv"],
        stats["GM"]["stdv"],
    )
    # M2WM
    results["wm2max"] = wm2max(data, stats["WM"]["median"])
    
    # ICVs
    results["icvs"] = volume_fraction(segmdata)

    # RPVE
    #results["rpve"] = rpve(pvmdata, segm2)
    
    # Image specs
    results["size"] = {
            "x": int(data.shape[0]),
            "y": int(data.shape[1]),
            "z": int(data.shape[2]),
        }
    results["spacing"] = {
            i: float(v) for i, v in zip(["x", "y", "z"], datanii.header.get_zooms()[:3])
        }    
    return results

In [ ]:
pvms

In [ ]:
print(eval_iqm(datanii, pvms, segm_map))
print()
print(eval_iqm(datanii,segm_map, segm_map))